In [ ]:
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from scipy.fft import fft
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/project

/content/drive/MyDrive/project


In [ ]:
L=8064
Fs=128
Features=np.zeros((25600,32,3*128))
Labels=list()
for subject_id in range(0,32):
    if subject_id % 10 ==0:
      print(subject_id)
    loaded_data=None
    file_name=None
    if subject_id<9:
        file_name= 'data_preprocessed_python/s0'+str(subject_id+1)+'.dat'
    else:
        file_name= 'data_preprocessed_python/s'+str(subject_id+1)+'.dat'
    with open(file_name, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        loaded_data = u.load()
    data=loaded_data['data']
    Labels.append(loaded_data['labels'])
    
    for i in range(len(data)):
        for k in range(3,61,3):
          for j in range(32):   
            x=np.array(data[i][j][k*128:(k+3)*128]) -np.array((data[i][j][:3*128]))
            x=zscore(x)
            Features[subject_id*800+i*20+int(k/3-1)][j]=x

0
10
20
30


In [ ]:

def DeepConvNet(nb_classes, Chans = 64, Samples = 256,
                dropoutRate = 0.5):

    # start the model
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(25, (1, 5), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(25, (Chans, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block1)
    block1       = Activation('elu')(block1)
    block1       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block1)
    block1       = Dropout(dropoutRate)(block1)
  
    block2       = Conv2D(50, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block2       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block2)
    block2       = Activation('elu')(block2)
    block2       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block2)
    block2       = Dropout(dropoutRate)(block2)
    
    block3       = Conv2D(100, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block2)
    block3       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block3)
    block3       = Activation('elu')(block3)
    block3       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block3)
    block3       = Dropout(dropoutRate)(block3)
    
    block4       = Conv2D(200, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block3)
    block4       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block4)
    block4       = Activation('elu')(block4)
    block4       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block4)
    block4       = Dropout(dropoutRate)(block4)
    
    flatten      = Flatten()(block4)
    
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)



In [ ]:
y=np.zeros(800*32)
for j in range(32):
  for i in range(40):
    if Labels[j][i][1]>5:
      y[j*800+i*20:j*800+(i+1)*20]=1

In [ ]:
def run(train_index,test_index,
        dropout_rate=0.5):



  Y=np_utils.to_categorical(y)

  d_size, chans, samples = Features.shape
  
  model=DeepConvNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = dropout_rate)

  # compile the model and set the optimizers
  model.compile(loss='categorical_crossentropy', optimizer='adam', 
                metrics = ['accuracy'])
  
  # count number of parameters in the model
  checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                                save_best_only=True)
  
  
  class_weights = {0:np.sum(y)/(len(y)-np.sum(y)), 1:1}
  
  fittedModel = model.fit(Features[train_index], Y[train_index], batch_size = 64, epochs =30 , 
                          verbose=2, validation_data=(Features[test_index], Y[test_index]),
                          callbacks=[checkpointer],class_weight=class_weights)
  probs       = model.predict(Features[test_index])
  preds       = probs.argmax(axis = -1)
  test = Y[test_index].argmax(axis=-1)
  acc = accuracy_score(test, preds)
  cm = confusion_matrix(test, preds)
  f1= f1_score(test, preds)
  return test, preds, acc,f1 , cm, model
  

In [ ]:
kf=KFold(n_splits=5,shuffle=True,random_state=0)

In [ ]:
train_index=list(kf.split(Features))[0][0]
test_index=list(kf.split(Features))[0][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 44s - loss: 0.8926 - accuracy: 0.5635 - val_loss: 0.6134 - val_accuracy: 0.6650

Epoch 00001: val_loss improved from inf to 0.61338, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 13s - loss: 0.8054 - accuracy: 0.6157 - val_loss: 0.5847 - val_accuracy: 0.6879

Epoch 00002: val_loss improved from 0.61338 to 0.58466, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 13s - loss: 0.7457 - accuracy: 0.6443 - val_loss: 0.5517 - val_accuracy: 0.7215

Epoch 00003: val_loss improved from 0.58466 to 0.55172, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 13s - loss: 0.7169 - accuracy: 0.6645 - val_loss: 0.5767 - val_accuracy: 0.6953

Epoch 00004: val_loss did not improve from 0.55172
Epoch 5/30
320/320 - 13s - loss: 0.6828 - accuracy: 0.6880 - val_loss: 0.5738 - val_accuracy: 0.7043

Epoch 00005: val_loss did not improve from 0.55172
Epoch 6/30
320/320 - 13s - loss: 0.6671 - accuracy: 0.7017 - val_loss: 0.5258 - val_accuracy: 0.7328

Epoch 00006: val_

In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 0, 0]),
 array([1, 1, 0, ..., 0, 0, 0]),
 0.8857421875,
 0.9022556390977444,
 array([[1835,  308],
        [ 277, 2700]]),
 <keras.engine.functional.Functional at 0x7fc85023cd50>)

In [ ]:
train_index=list(kf.split(Features))[1][0]
test_index=list(kf.split(Features))[1][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 15s - loss: 0.8827 - accuracy: 0.5641 - val_loss: 0.6215 - val_accuracy: 0.6533

Epoch 00001: val_loss improved from inf to 0.62149, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 13s - loss: 0.8009 - accuracy: 0.6194 - val_loss: 0.7260 - val_accuracy: 0.6271

Epoch 00002: val_loss did not improve from 0.62149
Epoch 3/30
320/320 - 13s - loss: 0.7425 - accuracy: 0.6450 - val_loss: 0.5829 - val_accuracy: 0.6818

Epoch 00003: val_loss improved from 0.62149 to 0.58291, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 13s - loss: 0.7030 - accuracy: 0.6760 - val_loss: 0.5718 - val_accuracy: 0.6939

Epoch 00004: val_loss improved from 0.58291 to 0.57175, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 13s - loss: 0.6790 - accuracy: 0.6921 - val_loss: 0.5324 - val_accuracy: 0.7191

Epoch 00005: val_loss improved from 0.57175 to 0.53244, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 13s - loss: 0.6554 - accuracy: 0.7106 - val_loss: 0.5343 -

In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 0, 0]),
 array([1, 1, 1, ..., 0, 0, 0]),
 0.8896484375,
 0.9057862264465567,
 array([[1839,  361],
        [ 204, 2716]]),
 <keras.engine.functional.Functional at 0x7fc816127390>)

In [ ]:
train_index=list(kf.split(Features))[2][0]
test_index=list(kf.split(Features))[2][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 15s - loss: 0.8800 - accuracy: 0.5723 - val_loss: 0.6409 - val_accuracy: 0.6594

Epoch 00001: val_loss improved from inf to 0.64085, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 13s - loss: 0.7956 - accuracy: 0.6206 - val_loss: 0.5918 - val_accuracy: 0.6820

Epoch 00002: val_loss improved from 0.64085 to 0.59181, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 13s - loss: 0.7352 - accuracy: 0.6527 - val_loss: 0.5676 - val_accuracy: 0.6949

Epoch 00003: val_loss improved from 0.59181 to 0.56761, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 13s - loss: 0.7080 - accuracy: 0.6708 - val_loss: 0.5460 - val_accuracy: 0.7277

Epoch 00004: val_loss improved from 0.56761 to 0.54598, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 13s - loss: 0.6789 - accuracy: 0.6873 - val_loss: 0.5143 - val_accuracy: 0.7555

Epoch 00005: val_loss improved from 0.54598 to 0.51433, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 13s - loss: 0.6549

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 0, 0]),
 array([0, 1, 0, ..., 1, 0, 0]),
 0.8888671875,
 0.9037062108647825,
 array([[1881,  334],
        [ 235, 2670]]),
 <keras.engine.functional.Functional at 0x7fc7b598a4d0>)

In [ ]:
train_index=list(kf.split(Features))[3][0]
test_index=list(kf.split(Features))[3][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 45s - loss: 0.8888 - accuracy: 0.5610 - val_loss: 0.5959 - val_accuracy: 0.6729

Epoch 00001: val_loss improved from inf to 0.59595, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 14s - loss: 0.8071 - accuracy: 0.6173 - val_loss: 0.6786 - val_accuracy: 0.5961

Epoch 00002: val_loss did not improve from 0.59595
Epoch 3/30
320/320 - 13s - loss: 0.7377 - accuracy: 0.6512 - val_loss: 0.5934 - val_accuracy: 0.6885

Epoch 00003: val_loss improved from 0.59595 to 0.59337, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 14s - loss: 0.7103 - accuracy: 0.6695 - val_loss: 0.6026 - val_accuracy: 0.6779

Epoch 00004: val_loss did not improve from 0.59337
Epoch 5/30
320/320 - 14s - loss: 0.6818 - accuracy: 0.6875 - val_loss: 0.5180 - val_accuracy: 0.7504

Epoch 00005: val_loss improved from 0.59337 to 0.51797, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 14s - loss: 0.6616 - accuracy: 0.7044 - val_loss: 0.5508 - val_accuracy: 0.7107

Epoch 00006: val_

In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 0, 0]),
 array([1, 1, 1, ..., 0, 0, 0]),
 0.8779296875,
 0.893725556878082,
 array([[1867,  247],
        [ 378, 2628]]),
 <keras.engine.functional.Functional at 0x7fcdffeb7c50>)

In [ ]:
train_index=list(kf.split(Features))[4][0]
test_index=list(kf.split(Features))[4][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 15s - loss: 0.8920 - accuracy: 0.5573 - val_loss: 0.8794 - val_accuracy: 0.5242

Epoch 00001: val_loss improved from inf to 0.87937, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 13s - loss: 0.7999 - accuracy: 0.6149 - val_loss: 0.6282 - val_accuracy: 0.6465

Epoch 00002: val_loss improved from 0.87937 to 0.62824, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 14s - loss: 0.7430 - accuracy: 0.6456 - val_loss: 0.5491 - val_accuracy: 0.7160

Epoch 00003: val_loss improved from 0.62824 to 0.54911, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 14s - loss: 0.7080 - accuracy: 0.6724 - val_loss: 0.6045 - val_accuracy: 0.6678

Epoch 00004: val_loss did not improve from 0.54911
Epoch 5/30
320/320 - 14s - loss: 0.6787 - accuracy: 0.6896 - val_loss: 0.5755 - val_accuracy: 0.6930

Epoch 00005: val_loss did not improve from 0.54911
Epoch 6/30
320/320 - 14s - loss: 0.6546 - accuracy: 0.7082 - val_loss: 0.5098 - val_accuracy: 0.7512

Epoch 00006: val_

In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 0, 0]),
 array([1, 1, 1, ..., 0, 0, 0]),
 0.8888671875,
 0.9048335842114065,
 array([[1846,  342],
        [ 227, 2705]]),
 <keras.engine.functional.Functional at 0x7fcdfff0d690>)